# libs, dataset

In [ ]:
!rm -rf used_cars

In [2]:
!git lfs clone https://huggingface.co/datasets/imbalet/used_cars

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Клонирование в «used_cars»...


remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 4 (from 1)
Распаковка объектов: 100% (18/18), 3.71 КиБ | 56.00 КиБ/с, готово.


In [ ]:
!pip install numpy pandas scikit-learn tensorflow

# import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

2024-12-05 23:45:48.600317: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 23:45:48.666989: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 23:45:48.846666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733431549.120913  169010 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733431549.202927  169010 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 23:45:49.595036: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Creating db
in progress...

In [2]:
df = pd.read_csv("used_cars/dataset.csv")

In [5]:
df[df["super_gen_name"].isna()]

,owners,year,price,region,mileage,mark,model,complectation,steering_wheel,gear_type,...,transmission,power,displacement,characteristics,color,body_type_type,body_type_name,super_gen_name,super_gen_year_from,super_gen_year_to
49,4,1976,600000,Свердловская область,200000,Lada (ВАЗ),2102,NaN,LEFT,REAR_DRIVE,...,MECHANICAL,59,1198.0,1.2 MT (59 л.с.),FF8649,WAGON_5_DOORS,Универсал 5 дв.,NaN,1971,1986.0
69,4,2011,350000,Белгородская область,208000,Lada (ВАЗ),2110,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,81,1596.0,Богдан 1.6 MT (81 л.с.),CACECB,SEDAN,Седан Богдан,NaN,1995,2014.0
82,4,2005,350000,Краснодарский край,200000,Lada (ВАЗ),2114,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,77,1499.0,1.5 MT (77 л.с.),007F00,HATCHBACK_5_DOORS,Хэтчбек 5 дв.,NaN,2001,2013.0
105,3,2005,150000,Волгоградская область,235000,ГАЗ,31105 «Волга»,NaN,LEFT,REAR_DRIVE,...,MECHANICAL,131,2286.0,2.3 MT (131 л.с.),CACECB,SEDAN,Седан,NaN,2003,2009.0
106,2,2001,150000,Красноярский край,190000,ГАЗ,3110 «Волга»,NaN,LEFT,REAR_DRIVE,...,MECHANICAL,131,2287.0,2.3 MT (131 л.с.),007F00,SEDAN,Седан,NaN,1997,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432206,1,2003,290000,Москва и Московская область,328000,Citroen,Xsara,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,109,1587.0,1.6 MT (109 л.с.),97948F,HATCHBACK_5_DOORS,Хэтчбек 5 дв.,NaN,1997,2006.0
432218,2,1984,290000,Москва и Московская область,55000,Lada (ВАЗ),2106,NaN,LEFT,REAR_DRIVE,...,MECHANICAL,64,1294.0,1.3 MT (64 л.с.),EE1D19,SEDAN,Седан,NaN,1976,2006.0
432219,4,2011,288000,Республика Дагестан,200000,Lada (ВАЗ),2114,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,81,1596.0,1.6 MT (81 л.с.),FAFBFB,HATCHBACK_5_DOORS,Хэтчбек 5 дв.,NaN,2001,2013.0
432224,2,2010,290000,Москва и Московская область,103630,ЗАЗ,Chance,S,LEFT,FORWARD_CONTROL,...,MECHANICAL,70,1299.0,1.3 MT (70 л.с.),EE1D19,HATCHBACK_5_DOORS,Хэтчбек 5 дв.,NaN,2009,2014.0


In [31]:
data = {}
for index, row in df.iterrows():
    mark = row["mark"]
    model = row["model"]
    gen = row["super_gen_name"]
    tr = row["transmission"]
    en = row["engine"]
    if mark not in data:
        data[mark] = {}
    if model not in data[mark]:
        data[mark][model] = {}
    if gen not in data[mark][model]:
        data[mark][model][gen] = {"transmission": set(), "engine": set(), "year": set(), "body_type": set(), "complectation": set()}
    data[mark][model][gen]["transmission"].add((str(tr), str(tr)))
    data[mark][model][gen]["engine"].add((str(en), str(en)))
    data[mark][model][gen]["year"].add((row["year"], row["year"]))
    data[mark][model][gen]["body_type"].add((row["body_type_type"], row["body_type_name"]))
    data[mark][model][gen]["complectation"].add((str(row["complectation"]), str(row["complectation"])))



In [15]:
data

{'Nissan': {'Note': {'III': {'transmission': {('AUTOMATIC', 'AUTOMATIC')},
    'engine': {('HYBRID', 'HYBRID')},
    'year': {(2020, 2020), (2021, 2021), (2022, 2022)},
    'body_type': {('ALLROAD_5_DOORS', 'Внедорожник 5 дв. Autech Crossover'),
     ('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв.'),
     ('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв. Aura'),
     ('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв. Aura Nismo')},
    'complectation': {(nan, nan)}},
   'II Рестайлинг': {'transmission': {('AUTOMATIC', 'AUTOMATIC'),
     ('MECHANICAL', 'MECHANICAL'),
     ('VARIATOR', 'VARIATOR')},
    'engine': {('GASOLINE', 'GASOLINE'), ('HYBRID', 'HYBRID')},
    'year': {(2016, 2016),
     (2017, 2017),
     (2018, 2018),
     (2019, 2019),
     (2020, 2020)},
    'body_type': {('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв.'),
     ('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв. Nismo')},
    'complectation': {(nan, nan)}},
   'I Рестайлинг': {'transmission': {('AUTOMATIC', 'AUTOMATIC'),
     ('MECHANICAL', 'MECHANICAL'),
     ('VARIATOR', '

In [32]:
for i in data:
    for j in data[i]:
        for k in data[i][j]:
            data[i][j][k]["transmission"] = list(data[i][j][k]["transmission"])
            data[i][j][k]["engine"] = list(data[i][j][k]["engine"])
            data[i][j][k]["year"] = list(data[i][j][k]["year"])
            data[i][j][k]["body_type"] = list(data[i][j][k]["body_type"])
            data[i][j][k]["complectation"] = list(data[i][j][k]["complectation"])

        print(data[i][j][k])

{'transmission': [('AUTOMATIC', 'AUTOMATIC'), ('VARIATOR', 'VARIATOR'), ('MECHANICAL', 'MECHANICAL')], 'engine': [('DIESEL', 'DIESEL'), ('GASOLINE', 'GASOLINE')], 'year': [(2007, 2007), (2008, 2008), (2006, 2006), (2005, 2005)], 'body_type': [('HATCHBACK_5_DOORS', 'Хэтчбек 5 дв.')], 'complectation': [('Comfort', 'Comfort'), ('Luxury', 'Luxury'), ('Comfort +condition', 'Comfort +condition'), ('nan', 'nan'), ('Luxury +climate-control', 'Luxury +climate-control'), ('Tekna', 'Tekna')]}
{'transmission': [('VARIATOR', 'VARIATOR')], 'engine': [('GASOLINE', 'GASOLINE')], 'year': [(2024, 2024)], 'body_type': [('ALLROAD_5_DOORS', 'Внедорожник 5 дв.')], 'complectation': [('Leading', 'Leading')]}
{'transmission': [('AUTOMATIC', 'AUTOMATIC'), ('VARIATOR', 'VARIATOR')], 'engine': [('GASOLINE', 'GASOLINE')], 'year': [(2009, 2009), (2006, 2006), (2008, 2008), (2005, 2005), (2007, 2007)], 'body_type': [('SEDAN', 'Седан')], 'complectation': [('nan', 'nan')]}
{'transmission': [('MECHANICAL', 'MECHANICAL'

In [10]:
for i in data:
    for j in data[i]:
        for k in data[i][j]:
            min_y = min( data[i][j][k]["year"])
            max_y = max( data[i][j][k]["year"])
            data[i][j][k]["year"] = [i for i in range(min_y, max_y + 1)]

TypeError: can only concatenate tuple (not "int") to tuple

In [16]:
df.columns

Index(['owners', 'year', 'price', 'region', 'mileage', 'mark', 'model',
       'complectation', 'steering_wheel', 'gear_type', 'engine',
       'transmission', 'power', 'displacement', 'characteristics', 'color',
       'body_type_type', 'body_type_name', 'super_gen_name',
       'super_gen_year_from', 'super_gen_year_to'],
      dtype='object')

In [33]:
cols = ["owners", 'region', 'steering_wheel', 'gear_type', "color"]
vars = {i: list(map(str, set(df[i]))) for i in cols}

In [34]:
vars["mark"] = data

In [36]:
import json
with open("variants_cars.json", 'w') as f:
    json.dump(data, f, ensure_ascii = False)
    

In [35]:
import json
with open("variants.json", 'w') as f:
    json.dump(vars, f, ensure_ascii = False)

# dataset


In [37]:
df = pd.read_csv("used_cars/dataset.csv")

In [43]:
df.sample(3)

,owners,year,price,region,mileage,mark,model,complectation,steering_wheel,gear_type,...,transmission,power,displacement,characteristics,color,body_type_type,body_type_name,super_gen_name,super_gen_year_from,super_gen_year_to
284126,4,2003,110000,Липецкая область,333333,Lada (ВАЗ),2112,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,92,1500.0,1.5 MT (92 л.с.),97948F,HATCHBACK_5_DOORS,Хэтчбек 5 дв.,NaN,1998,2009.0
105042,1,2023,6790000,Москва и Московская область,93,Hyundai,Santa Fe,Calligraphy,LEFT,ALL_WHEEL_DRIVE,...,ROBOT,281,2497.0,2.5 AMT (281 л.с.) 4WD,040001,ALLROAD_5_DOORS,Внедорожник 5 дв.,V (MX5),2023,2024.0
208994,4,2012,870000,Москва и Московская область,163000,Skoda,Octavia,NaN,LEFT,FORWARD_CONTROL,...,MECHANICAL,102,1595.0,1.6 MT (102 л.с.),FAFBFB,LIFTBACK,Лифтбек,II (A5) Рестайлинг,2008,2013.0


In [44]:
df.columns

Index(['owners', 'year', 'price', 'region', 'mileage', 'mark', 'model',
       'complectation', 'steering_wheel', 'gear_type', 'engine',
       'transmission', 'power', 'displacement', 'characteristics', 'color',
       'body_type_type', 'body_type_name', 'super_gen_name',
       'super_gen_year_from', 'super_gen_year_to'],
      dtype='object')

In [38]:
df = df.drop(["body_type_name", "super_gen_year_from", "super_gen_year_to", "characteristics"], axis = 1)

In [46]:
df.dtypes

owners              int64
year                int64
price               int64
region             object
mileage             int64
mark               object
model              object
complectation      object
steering_wheel     object
gear_type          object
engine             object
transmission       object
power               int64
displacement      float64
color              object
body_type_type     object
super_gen_name     object
dtype: object

In [39]:
for i in df.columns:
    print(i, len(df[df[i].isna()]))

owners 0
year 0
price 0
region 0
mileage 0
mark 0
model 0
complectation 260331
steering_wheel 0
gear_type 0
engine 0
transmission 0
power 0
displacement 2757
color 0
body_type_type 0
super_gen_name 44115


In [48]:
df[df["displacement"].isna()] = 0
df['complectation'] = df.apply(lambda row: row['mark'] + row['model'] if pd.isna(row['complectation']) else row['complectation'], axis=1)
df['super_gen_name'] = df.apply(lambda row: row['mark'] + row['model'] if pd.isna(row['super_gen_name']) else row['super_gen_name'], axis=1)

In [49]:

df["year"] = df["year"].astype("object")
# df["owners"] = df["owners"].astype("object")

In [50]:
df.dtypes

owners              int64
year               object
price               int64
region             object
mileage             int64
mark               object
model              object
complectation      object
steering_wheel     object
gear_type          object
engine             object
transmission       object
power               int64
displacement      float64
color              object
body_type_type     object
super_gen_name     object
dtype: object

In [51]:
num_features = ["owners", "mileage", "power", "displacement"]
cat_features = df.select_dtypes(include=['object']).columns

In [52]:
price = ["price"]

price_scaler = StandardScaler()

df[price] = price_scaler.fit_transform(df[price])

In [53]:
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])

In [54]:
label_encoders = {}
for feature in cat_features:
    le = LabelEncoder()
    df[feature] = list(map(str, df[feature]))
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le


In [55]:
# creating config scale/encoders file
import json
scalers = {scaler.feature_names_in_[i]: (float(scaler.mean_[i]), float(scaler.scale_[i])) for i in range(len(scaler.mean_))}
scalers["price"] = (float(price_scaler.mean_[0]), float(price_scaler.scale_[0]))

labels = {i: list(map(str, label_encoders[i].classes_)) for i in label_encoders}

with open("configs.json", 'w', encoding="utf-8") as f:
    json.dump({"scalers": scalers, "labels": labels}, f, ensure_ascii=False, indent=4)

In [56]:
df.sample(3)

,owners,year,price,region,mileage,mark,model,complectation,steering_wheel,gear_type,engine,transmission,power,displacement,color,body_type_type,super_gen_name
128709,1.290019,72,-0.884616,28,0.972664,93,76,3290,1,2,2,2,-0.794615,-0.486347,10,11,1360
240888,-1.209993,82,0.513630,35,0.340832,118,1394,2479,1,1,2,1,0.377055,1.345829,3,2,1275
51981,0.456682,81,-0.185493,73,-0.220332,122,1081,4824,1,2,2,4,-0.435125,-0.485015,12,2,491


In [57]:
X = df.drop(columns=['price'])
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model


In [58]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

/home/imbalet/nn/nn_project/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-05 01:46:20.546945: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [59]:
model.compile(optimizer='adam', loss = "mean_squared_error")

In [60]:
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=64)

Epoch 1/50
1110/4880 ━━━━━━━━━━━━━━━━━━━━ 1:31 24ms/step - loss: 314.1954

KeyboardInterrupt: 

In [ ]:
model.save('model.keras')

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss (MSE): {test_loss}')

In [ ]:
original_data = price_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))

In [ ]:
pred_unscaled = price_scaler.inverse_transform(model.predict(X_test))

In [ ]:
# MAE
sum(abs(original_data - pred_unscaled)) / original_data.shape[0]